# Báo cáo tiểu luận cuối kì môn Thống kê Ứng dụng

**Người thực hiện: Vũ Quang Hải**

**MSV: 16003499**

Các nguồn dữ liệu

1. Thống kê giấy phép buôn bán vũ khí hiện hành liên bang Mĩ (tới tháng 7 năm 2017)

Tác giả: Cục Rượu, Thuốc lá, Súng và Chất nổ Hoa Kì
URL: [Listing of Federal Firearms Licensees](https://www.atf.gov/firearms/listing-federal-firearms-licensees-ffls-2017)

2. Thống kê các vụ va chạm giữa máy bay do động vật

Tác giả: Nhiều tác giả, liệt kê trong nghiên cứu dưới đây
URL: [Wildlife strikes to civil aircraft in the United States, 1990-2014](https://www.researchgate.net/publication/291353952_Wildlife_strikes_to_civil_aircraft_in_the_United_States_1990-2014)

3. Thống kê Bất động sản tại Ames, Hoa Kì

Tác giả: Dead De Cock, Đại học bang Truman
URL: [Ames Housing Data](http://jse.amstat.org/v19n3/decock.pdf)

#  1. Bài toán kiểm định số lượng cửa hàng bán súng tại các quận trong bang ở Mĩ.

Dưới đây là dữ liệu minh hoạ:

In [ ]:
import pandas as pd
import numpy as np

licensees = pd.read_csv("../input/federal-firearm-licensees/federal-firearm-licensees.csv", index_col=0)[1:]
licensees.head()

Giả sử ta muốn tính số lượng súng trung bình của từng quận. Dưới đây là biểu đồ tần số số lượng cửa hàng buôn bán súng theo từng quận. Để không tốn công chuẩn hoá dữ liệu, ta sẽ quy hoạch theo [ZIP Code](https://en.wikipedia.org/wiki/ZIP_Code), thay vì theo tên của quận.

*Chú ý: Cách tính này không bao hàm các quận không có cửa hàng bán súng, vậy nên dữ liệu này chỉ thống kê đối với các quận có cửa hàng bán súng*

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
licensees['Premise Zip Code'].value_counts().plot()

Ta tính trung bình số lượng cửa hàng súng trung bình:

In [ ]:
licensees['Premise Zip Code'].value_counts().mean()

Để minh hoạ, dưới đây là biểu đồ lấy dữ liệu ngẫu nhiên với kích thước mẫu tăng dần

In [ ]:
r = (licensees['Premise Zip Code']
         .value_counts()
         .sample(len(licensees['Premise Zip Code'].unique()) - 1))
pd.Series(r.cumsum() / np.array(range(1, len(r) + 1))).reset_index(drop=True).plot.line(
    figsize=(12, 4), linewidth=1
)

Với số lượng cửa hàng trung bình trên, ta đặt giả thiết ước lượng kì vọng như sau:

$$H_0: \mu = 2.75 $$
$$H_1: \mu \neq 2.75 $$

Giả sử mức rủi ro là 5%

Ta test thống kê dựa trên dữ liệu đã chọn như sau:

In [ ]:
import scipy.stats as stats

countyFreq = licensees['Premise Zip Code'].value_counts()
stats.ttest_1samp(a=countyFreq, popmean=2.75)

Như vậy, ta có độ chệch là xấp xỉ 0.36, p-value = 0.72 > 0.05, ta bác bỏ $H_1$, chấp nhận $H_0$, tức có xấp xỉ tận 2 đến 3 cửa hàng súng mỗi quận ở Mĩ.

# 2. Bài toán so sánh tham số của hai hoặc nhiều tổng thể

Ta thực hiện bài toán so sánh các quận của các bang khác nhau về số cửa hàng bán súng, xem thực tế các bang có số cửa hàng bán súng theo quận có như nhau hay không

Dưới ta sẽ lấy ví dụ ở 2 bang nổi tiếng: Texas (TX) và New York (NY)

In [ ]:
newYorkLicensees = licensees[(licensees['Premise State'] == 'NY')]
texasLicensees = licensees[(licensees['Premise State'] == 'TX')]

Nhân tiện, ta thử kiểm định số lượng cửa hàng bán súng tính theo các quận của hai bang trên so với trung bình cả nước Mĩ với mức rủi ro 5%

In [ ]:
nyCountyFreq = newYorkLicensees['Premise Zip Code'].value_counts()
txCountyFreq = texasLicensees['Premise Zip Code'].value_counts()

nyMean = nyCountyFreq.mean()
txMean = txCountyFreq.mean()

nyResult = stats.ttest_1samp(a=nyCountyFreq, popmean=2.75)
txResult = stats.ttest_1samp(a=txCountyFreq, popmean=2.75)

print("New York mean: ", nyMean)
print("Texas mean: ", txMean)

print("New York result: ", nyResult)
print("Texas result: ", txResult)

Ta thấy rằng thực tế hai bang này không có số cửa hàng bán súng khác với số liệu toàn nước Mĩ.

Giờ ta kiểm thử giữa hai bang cùng với mức rủi ro 5%, Thiết lập bài toán ước lượng kì vọng như sau:

$$H_0: \mu_{ny} = \mu_{tx} $$
$$H_1: \mu_{ny} \neq \mu_{tx} $$

In [ ]:
stats.ttest_ind(nyCountyFreq, txCountyFreq)

Dễ thấy p-value = $2.07e^{-46} < 0.05$, tức hai quận này hoàn toàn chẳng giống nhau về phân bố cửa hàng bán súng theo quận.

Để dễ theo dõi, ta có hai biểu đồ như sau:


In [ ]:
nyCountyFreq.plot()

In [ ]:
txCountyFreq.plot()

Ta thấy dáng hình phân bố của bang New York (đường màu đỏ) thoải hơn khá nhiều so với bang Texas (đường màu xanh).

Để so sánh, ta sẽ chọn một bang khác của nước Mĩ là Arkansas (AR). Cùng với thiết lập bài toán như trên, ta tính được như sau:

In [ ]:
arkansasLicensees = licensees[(licensees['Premise State'] == 'AR')]
arCountyFreq = arkansasLicensees['Premise Zip Code'].value_counts()

stats.ttest_ind(arCountyFreq, txCountyFreq)

Qua kết quả này, ta thấy p-value = 0.69 > 0.05, tức phân bố các cửa hàng súng ở các quận ở 2 bang này là tương đối giống nhau. Theo dõi qua biểu đồ sau để đối chiếu với bang Texas:

In [ ]:
arCountyFreq.plot()

# 3. Bài toán tìm phân bố các vụ tai nạn do va chạm giữa chim chóc và máy bay

Dưới đây là dữ liệu minh hoạ

In [ ]:
strikes = pd.read_csv("../input/wildlife-strikes/database.csv", index_col=0)
strikes.head()

Dưới đây là đồ thị minh hoạ phân bố tai nạn theo các tháng trong năm

In [ ]:
strikeByMonths = strikes['Incident Month']
strikesFreq = strikeByMonths.value_counts().sort_index()
observed = strikesFreq.values
totalObserved = sum(observed)

plt.plot(observed)

Từ dữ liệu ở trên, ta ước lượng $\mu$ và $\sigma$ để sinh phân phối chuẩn

In [ ]:
var = strikeByMonths.var()
mu = strikeByMonths.mean()
sigma = np.sqrt(var)
print("variance=" + str(var), "mean=" + str(mu))

Từ đây ta tính được một phân phối chuẩn như sau:
 
$$N(\mu=7.78494854349952, \sigma^2=7.171839819877774)$$

Để minh hoạ cho dữ liệu thu được và dữ liệu dự đoán theo phân bố, ta có biểu đồ minh hoạ sau

In [ ]:
ran = np.random.normal(mu, np.sqrt(var), len(strikes))
x = np.arange(start=0, stop=12)
density = stats.gaussian_kde(ran)
expected = totalObserved * density(x)
plt.plot(x, observed, expected)

Biểu đồ giữa thực tế và dự đoán tương đối ngang nhau. Để so sánh hai giá trị này, ta sẽ sử dụng tiêu chuẩn bình phương:

In [ ]:
stats.chisquare(observed, expected)

Dữ liệu p-value = 0, khá vô lý do mâu thuẫn với định lý giới hạn trung tâm, khi dữ liệu thu được lại không hội tụ về phân phối chuẩn. Nhưng khi xem lại công thức tính tiêu chuẩn bình phương:

$$\chi^2=\frac{1}{d}\sum_{k=1}^{n} \frac{(O_k - E_k)^2}{E_k}$$

Việc p-value thu được bằng 0 là dễ hiểu do các lý do sau:
* Dữ liệu quá thưa so với kích thước mẫu
* Kích thước mẫu quá lớn
* Công thức tính làm luỹ thừa 2 độ chênh lệch

Như trên, ta có thể kết luận được rằng, ta không nên sử dụng phương pháp Tiêu chuẩn bình phương để tính toán trên những bộ dữ liệu quá thưa và có không gian mẫu quá lớn. Để khách quan, ta sử dụng một phương pháp pháp khác để xét tính tương quan là Kolmogorov–Smirnov:

In [ ]:
stats.kstest(observed, expected)


Lần này ta thu được p-value hợp lý hơn: 0.87, chưa thực sự tốt để có thể kết luận rằng dữ liệu trên tuân theo phân bố chuẩn, nhưng đủ tốt để thấy độ tương quan giữa phân bố chuẩn và dữ liệu thu được. Để cải thiện dữ liệu trên, ta cần khắc phục các vấn đề sau:
* Dữ liệu cần phải dầy hơn (ví dụ như thống kê theo từng tuần của tháng)

Như vậy sẽ cho ra một dữ liệu chi tiết hơn.

Để phục vụ mục đích biểu diễn, ta sẽ thử phương pháp nội suy Lagrange để nội suy một đa thức 11 bậc để sinh ra hàm đại diện cho dữ liệu ban đâu:

In [ ]:
poly = np.poly1d(np.polyfit(x=np.arange(0, 12) , y=observed, deg=11))
plt.plot(poly(x))

Sau đó ta thử phép thử K-S ở trên trên hàm số vừa tạo ra với bước dữ liệu, 

In [ ]:
smallRange = np.arange(0, 11.1, 0.1)
largeRange = np.arange(0, 11.1, 0.5)

print(stats.kstest(poly(smallRange), totalObserved * density(smallRange)))
print(stats.kstest(poly(largeRange), totalObserved * density(largeRange)))

Kết quả ở trên tương ứng với dữ liệu dầy gấp 10 dữ liệu ban đầu, kết quả ở dưới tương ứng với dữ liệu dầy gấp 2 dữ liệu ban đâu. Từ dữ liệu p-value cho ta thấy rằng việc xấp xỉ hàm số không thực sự giúp ích gì nếu dữ liệu đã vốn thưa.

# 4. Bài toán dự đoán giá Bất động sản

Dựa vào dữ liệu trên, ta sẽ thực hiện bài toán dự đoán giá bất động sản tại Ames, Iowa sử dụng phương pháp hồi quy tuyến tính.

Dưới đây ta sẽ minh hoạ dữ liệu:

In [ ]:
import seaborn as sns
%matplotlib inline

plt.style.use("fivethirtyeight")
sns.set_style("whitegrid")

USAhousing = pd.read_csv('/kaggle/input/usa-housing/USA_Housing.csv')
USAhousing.head()

Ta sẽ tách dữ liệu ra thành hai phần: Dữ liệu thử và dữ liệu dùng để sinh hồi quy tuyến tính.

Ta sẽ chọn cột giá tiền để hồi quy còn các cột dữ liệu số còn lại dùng để làm dữ liệu tiên đoán:

In [ ]:
X = USAhousing[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
               'Avg. Area Number of Bedrooms', 'Area Population']]
y = USAhousing['Price']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

Để dữ liệu phù hợp với việc hồi quy tuyến tính, ta nên chuẩn hoá dữ liệu về gần các điều kiện này nhất có thể:
* Tuyến tính hoá dữ liệu: Bỏ các phụ thuộc trong dữ liệu
* Khử nhiễu: Dữ liệu đưa vào mô hình hồi quy tuyến tính nên là dữ liệu trơn để không sinh ra quá nhiều sai sót
* Cố gắng đẩy về dạng phân bố chuẩn: Theo kinh nghiệm, đưa dữ liệu về phân bố chuẩn sẽ sinh ra mô hình hồi quy đáng tin cậy hơn

Dưới đây ta sẽ sử dụng các công cụ có sẵn để thực hiện các bước trên:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

Sau đó, ta đẩy dữ liệu vào mô hình hồi quy tuyến tính:

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train,y_train)

Cuối cùng là dự đoán dữ liệu từ mô hình hồi quy trên:

In [ ]:
pred = lin_reg.predict(X_test)


plt.scatter(y_test, pred)

In [ ]:
Đồ thị phân bố chênh lệch:

In [ ]:
sns.distplot((y_test - pred), bins=50);